In [16]:
import pandas as pd
import datetime

In [258]:
df_snow = pd.read_excel("data/Климат Средние величины.xlsx")

In [259]:
df_snow

,id_station,station,1,2,3,4,5,6,7,8,9,10,11,12,start,end
0,24817,Ербогачен,15.0,17.0,31.0,46.0,48.0,31.0,30.0,37.0,46.0,40.0,29.0,19.0,2022-10-11,2022-04-21
1,24918,Преображенка,17.0,21.0,35.0,54.0,52.0,27.0,24.0,34.0,44.0,40.0,30.0,21.0,2022-10-13,2022-04-15
2,29393,Червянка,23.0,25.0,40.0,54.0,49.0,30.0,39.0,47.0,48.0,49.0,45.0,32.0,2022-10-18,2022-03-31
3,29498,Шиткино,26.0,33.0,45.0,54.0,74.0,51.0,59.0,60.0,48.0,51.0,53.0,44.0,2022-10-18,2022-03-24
4,29590,Новочунка,27.0,34.0,44.0,53.0,70.0,38.0,46.0,57.0,48.0,50.0,55.0,44.0,2022-10-19,2022-03-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,30812,Култук,14.0,17.0,39.0,65.0,71.0,77.0,81.0,97.0,59.0,38.0,30.0,22.0,2022-10-27,2022-03-30
65,30815,Хамар-Дабан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
66,30818,Байкальск,12.0,13.0,28.0,47.0,55.0,55.0,47.0,71.0,44.0,29.0,24.0,18.0,2022-10-19,2022-04-10
67,UIBB,Братск Аэропорт,24.0,26.0,31.0,37.0,43.0,30.0,41.0,54.0,37.0,35.0,40.0,32.0,2022-10-18,2022-03-31


In [304]:
def changing_columns():
    cols = list(df_temp.columns) 
    cols.append('temp')
    df_temp.reset_index(inplace = True)
    df_temp.columns = cols
    df_temp.drop(columns = 'temp', inplace = True) 

def processing_date():
    df_temp.rename(columns = {df_temp.columns[0]:'datetime'}, inplace = True)
    df_temp[['date', 'time']] = df_temp['datetime'].astype(str).str.split(' ', expand = True)
    df_temp['datetime'] = pd.to_datetime(df_temp['datetime'], format = "%d.%m.%Y %H:%M")
    df_temp['date'] = pd.to_datetime(df_temp['date'], format = "%d.%m.%Y")
    df_temp['month'] = df_temp.date.dt.month
    df_temp['year'] = df_temp.date.dt.year
    
    start, end = df_snow.loc[df_snow.id_station==s].set_index('id_station').iloc[0][-2:]
    df_temp['start'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = start.day))
    df_temp['end'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, month = end.month, day = end.day))
    return (start, end)

def get_snowiness(df_group):
    df_winter = df_group.reset_index().pivot(columns = 'month', index = ['start', 'end'], values = 'RRR')
    months = df_winter.columns.to_list()
    df_winter['sum_RRR_fact'] = df_winter.sum(1)
    statistic = df_snow.loc[df_snow.id_station==s].set_index('id_station').iloc[0][months].sum()
    df_winter['percent'] = (df_winter.sum_RRR_fact/statistic*100)-100

    df_w = df_winter.copy()
    df_w[start.month] = round(31-start.day, -1)//10/3*df_w[start.month]
    df_w[end.month] = round(end.day, -1)//10/3*df_w[end.month]
    df_w['decade_sum_RRR'] = df_w[months].sum(1)
    df_w['decade_percent'] = (df_w.decade_sum_RRR/statistic*100)-100

    df_winter['decade_start'] = df_w[start.month]
    df_winter['decade_end'] = df_w[end.month]
    df_winter['start_month'] = start.month
    df_winter['end_month'] = end.month
    df_winter['sum_RRR'] = statistic
    df_winter['decade_sum_RRR'] = df_w['decade_sum_RRR']
    df_winter['decade_percent'] = df_w['decade_percent']
    
    return df_winter

errors_station = df_snow.loc[(df_snow.id_station.isna()) | (df_snow.start.isna())].id_station.to_list()
df_snow['id_station'] = df_snow.id_station.astype(str)
dfs = []
datalist = df_snow.loc[(~df_snow.id_station.isna()) & ~(df_snow.start.isna())].id_station
for s in notebook.tqdm(datalist, total = len(datalist)):
    names = ['.01.01.2017.31.12.2020.1.0.0.ru.utf8.00000000','.26.02.2020.31.12.2020.1.0.0.ru.utf8.00000000', '.30.10.2017.31.12.2020.1.0.0.ru.utf8.00000000']
    for n in names:
        try:
#             print('data/'+str(s)+n+'.csv')
            df_temp = pd.read_csv('data/'+s+n+'.csv', skiprows = 6, sep = ';')
            break
        except:
            continue
        else:
            break
    else:
        print(s, 'not found file')
        errors_station.append(s)

    if s not in errors_station:
         
        changing_columns()

        df_temp['RRR'] = df_temp['RRR'].replace('Осадков нет', 0)
        df_temp['RRR'] = df_temp['RRR'].replace('Следы осадков', 0).astype(float)

        start, end = processing_date()

        dates = df_temp.loc[(df_temp.WW.str.contains('снег')) | (df_temp.WW.str.contains('снеж')) |
                            (df_temp.W1.astype(str).str.contains('снег')) | (df_temp.W1.astype(str).str.contains('снеж')) |
                            (df_temp.W2.astype(str).str.contains('снег')) | (df_temp.W2.astype(str).str.contains('снеж')) |
                            (df_temp["E'"].astype(str).str.contains('снег')) | (df_temp["E'"].astype(str).str.contains('снеж')) |
                            (df_temp["sss"].astype(str).str.contains('снег')) | (df_temp["sss"].astype(str).str.contains('снеж')) |
                            (~df_temp.sss.isna())][['date']].drop_duplicates()['date']
        try:
            df_group = df_temp.loc[(df_temp.date.apply(lambda x: (x>=pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = 1)) \
                                    and (x<=pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, month = end.month, day = 1) ))) 
                    & (df_temp.date.isin(dates))\
                    & (df_temp.date >= pd.Timestamp(df_temp.date.dt.year.min(), start.month, 1)) \
                    & (df_temp.date <= pd.Timestamp(df_temp.date.dt.year.max(), end.month, (datetime.date(df_temp.date.dt.year.max(), end.month+1, 1) - datetime.timedelta(days=1)).day))
                   ].sort_values('date')\
                    .groupby(['start', 'end', 'month']).sum()[['RRR']]
            
            df_winter = get_snowiness(df_group)
            df_winter['id_station'] = s

            dfs.append(df_winter.reset_index())
        except Exception as e:
            print(s, e, 'len(dates): ', len(dates), dates.min(), dates.max())
            errors_station.append(s)
    

29590 10 len(dates):  125 2020-02-27 00:00:00 2020-12-30 00:00:00
30317 4 len(dates):  170 2017-04-07 00:00:00 2020-12-18 00:00:00
30507 10 len(dates):  114 2020-02-27 00:00:00 2020-12-30 00:00:00
30514 10 len(dates):  112 2020-02-27 00:00:00 2020-12-30 00:00:00
30712 3 len(dates):  180 2017-10-12 00:00:00 2020-12-26 00:00:00
nan not found file
30791 11 len(dates):  0 NaT NaT
UIBB not found file
nan not found file



In [305]:
errors_station

['29794',
 '29892',
 '30439',
 '30537',
 '30632',
 '30815',
 '29590',
 '30317',
 '30507',
 '30514',
 '30712',
 'nan',
 '30791',
 'UIBB',
 'nan']

In [306]:
df = pd.concat(dfs, ignore_index = True)

In [307]:
len(df.id_station.drop_duplicates())

54

In [308]:
df = df.merge(df_snow[['id_station', 'station']], how = 'left', on = 'id_station')

In [309]:
df

,start,end,1,2,3,4,10,11,12,sum_RRR_fact,percent,decade_start,decade_end,start_month,end_month,sum_RRR,decade_sum_RRR,decade_percent,id_station,station
0,2017-10-11,2018-04-21,18.0,18.6,14.1,0.0,16.6,43.0,26.5,136.8,-30.558376,11.066667,0.000000,10,4,197.0,131.266667,-33.367174,24817,Ербогачен
1,2018-10-11,2019-04-21,25.5,21.0,31.1,0.0,23.7,23.2,28.4,152.9,-22.385787,15.800000,0.000000,10,4,197.0,145.000000,-26.395939,24817,Ербогачен
2,2019-10-11,2020-04-21,21.6,21.3,23.9,0.0,43.8,18.9,20.6,150.1,-23.807107,29.200000,0.000000,10,4,197.0,135.500000,-31.218274,24817,Ербогачен
3,2017-10-13,2018-04-15,24.0,14.6,13.1,0.3,20.9,29.1,33.3,135.3,-37.935780,13.933333,0.200000,10,4,218.0,128.233333,-41.177370,24918,Преображенка
4,2018-10-13,2019-04-15,22.3,19.0,15.9,0.0,19.9,16.3,27.7,121.1,-44.449541,13.266667,0.000000,10,4,218.0,114.466667,-47.492355,24918,Преображенка
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2018-10-27,2019-03-30,3.1,0.8,NaN,NaN,8.5,6.6,0.8,19.8,-87.625000,0.000000,NaN,10,3,160.0,11.300000,-92.937500,30812,Култук
152,2019-10-27,2020-03-30,1.3,6.6,0.3,NaN,1.0,4.6,1.9,15.7,-90.187500,0.000000,0.300000,10,3,160.0,14.700000,-90.812500,30812,Култук
153,2017-10-19,2018-04-10,21.0,17.7,21.2,5.0,0.7,74.7,32.3,172.6,0.935673,0.233333,1.666667,10,4,171.0,168.800000,-1.286550,30818,Байкальск
154,2018-10-19,2019-04-10,13.4,3.9,28.4,NaN,32.8,21.0,25.6,125.1,-26.842105,10.933333,NaN,10,4,171.0,103.233333,-39.629630,30818,Байкальск


In [311]:
df.loc[df.decade_percent>-25]

,start,end,1,2,3,4,10,11,12,sum_RRR_fact,percent,decade_start,decade_end,start_month,end_month,sum_RRR,decade_sum_RRR,decade_percent,id_station,station
83,2017-10-17,2018-04-15,21.1,9.0,5.3,0.0,20.0,44.0,12.0,111.4,-10.880000,6.666667,0.000000,10,4,125.0,98.066667,-21.546667,30433,Нижнеангарск
85,2019-10-17,2020-04-15,22.8,23.2,29.1,0.0,59.6,15.9,3.5,154.1,23.280000,19.866667,0.000000,10,4,125.0,114.366667,-8.506667,30433,Нижнеангарск
86,2017-10-14,2018-04-15,17.9,18.4,15.9,0.9,4.0,11.4,51.7,120.2,-22.948718,2.666667,0.600000,10,4,156.0,118.566667,-23.995726,30437,Карам
98,2017-10-22,2018-03-31,17.9,16.6,1.0,NaN,8.9,23.8,62.8,131.0,-12.080537,2.966667,1.000000,10,3,149.0,125.066667,-16.062640,30603,Зима
122,2019-11-05,2020-03-23,1.4,3.0,1.0,NaN,NaN,3.4,49.0,57.8,7.037037,3.400000,0.666667,11,3,54.0,57.466667,6.419753,30637,Узуры
123,2017-11-04,2018-03-23,18.1,22.7,0.9,NaN,NaN,30.2,45.5,117.4,23.578947,30.200000,0.600000,11,3,95.0,117.100000,23.263158,30703,Инга
126,2017-11-04,2018-03-23,18.8,19.0,0.2,NaN,NaN,15.5,50.4,103.9,-10.431034,15.500000,0.133333,11,3,116.0,103.833333,-10.488506,30710,Иркутск
132,2017-10-30,2018-03-24,14.1,23.9,0.5,NaN,9.2,30.0,23.9,101.6,-16.033058,0.000000,0.333333,10,3,121.0,92.233333,-23.774105,30714,Хайта
135,2017-11-01,2018-03-23,12.7,19.1,1.0,NaN,NaN,14.3,48.6,95.7,11.279070,14.300000,0.666667,11,3,86.0,95.366667,10.891473,30715,Ангарск
153,2017-10-19,2018-04-10,21.0,17.7,21.2,5.0,0.7,74.7,32.3,172.6,0.935673,0.233333,1.666667,10,4,171.0,168.800000,-1.286550,30818,Байкальск


In [278]:
df.to_csv('data/snowiness.csv', index = False)

# Проверки

In [279]:
df_temp = pd.read_csv('data/30712.01.01.2017.31.12.2020.1.0.0.ru.utf8.00000000.csv', skiprows = 6, sep = ';')

In [280]:
cols = list(df_temp.columns) 
cols.append('temp')
df_temp.reset_index(inplace = True)
df_temp.columns = cols
df_temp.drop(columns = 'temp', inplace = True) 

In [281]:
df_temp.rename(columns = {df_temp.columns[0]:'datetime'}, inplace = True)

In [282]:
df_temp[['date', 'time']] = df_temp['datetime'].astype(str).str.split(' ', expand = True)
df_temp['datetime'] = pd.to_datetime(df_temp['datetime'], format = "%d.%m.%Y %H:%M")
df_temp['date'] = pd.to_datetime(df_temp['date'], format = "%d.%m.%Y")

In [283]:
df_temp['month'] = df_temp.date.dt.month
df_temp['year'] = df_temp.date.dt.year

In [287]:
df_temp['RRR'] = df_temp['RRR'].replace('Осадков нет', 0)
df_temp['RRR'] = df_temp['RRR'].replace('Следы осадков', 0).astype(float)
start, end = df_snow.loc[df_snow.id_station==s].set_index('id_station').iloc[0][-2:]
df_temp['start'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = start.day))
df_temp['end'] = df_temp.date.apply(lambda x: pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, month = end.month, day = end.day))


In [302]:
dates = df_temp.loc[(df_temp.WW.str.contains('снег')) | (df_temp.WW.str.contains('снеж')) |
            (df_temp.W1.astype(str).str.contains('снег')) | (df_temp.W1.astype(str).str.contains('снеж')) |
            (df_temp.W2.astype(str).str.contains('снег')) | (df_temp.W2.astype(str).str.contains('снеж')) |
            (df_temp["E'"].astype(str).str.contains('снег')) | (df_temp["E'"].astype(str).str.contains('снеж')) |
            (df_temp["sss"].astype(str).str.contains('снег')) | (df_temp["sss"].astype(str).str.contains('снеж')) |
            (~df_temp.sss.isna())][['date']].drop_duplicates()['date']

df_group = df_temp.loc[(df_temp.date.apply(lambda x: (x>=pd.Timestamp(year = x.year-1 if x.month < 9 else x.year, month = start.month , day = 1)) \
                                    and (x<=pd.Timestamp(year = x.year+1 if x.month > 9 else x.year, month = end.month, day = 1) ))) 
                    & (df_temp.date.isin(dates))\
                    & (df_temp.date >= pd.Timestamp(df_temp.date.dt.year.min(), start.month, 1)) \
                    & (df_temp.date <= pd.Timestamp(df_temp.date.dt.year.max(), end.month, (datetime.date(df_temp.date.dt.year.max(), end.month+1, 1) - datetime.timedelta(days=1)).day))
                   ].sort_values('date')\
                    .groupby(['start', 'end', 'month']).sum()[['RRR']]
            
df_winter = df_group.reset_index().pivot(columns = 'month', index = ['start', 'end'], values = 'RRR')
months = df_winter.columns.to_list()
df_winter['sum_RRR_fact'] = df_winter.sum(1)
statistic = df_snow.loc[df_snow.id_station==s].set_index('id_station').iloc[0][months].sum()
df_winter['percent'] = (df_winter.sum_RRR_fact/statistic*100)-100

df_w = df_winter.copy()
df_w[start.month] = round(31-start.day, -1)//10/3*df_w[start.month]
df_w[end.month] = round(end.day, -1)//10/3*df_w[end.month]
df_w['decade_sum_RRR'] = df_w[months].sum(1)
df_w['decade_percent'] = (df_w.decade_sum_RRR/statistic*100)-100

df_winter['decade_start'] = df_w[start.month]
df_winter['decade_end'] = df_w[end.month]
df_winter['start_month'] = start.month
df_winter['end_month'] = end.month
df_winter['sum_RRR'] = statistic
df_winter['decade_sum_RRR'] = df_w['decade_sum_RRR']
df_winter['decade_percent'] = df_w['decade_percent']
df_winter['id_station'] = s


In [303]:
df_winter

,month,1,2,3,4,10,11,12,sum_RRR_fact,percent,decade_start,decade_end,start_month,end_month,sum_RRR,decade_sum_RRR,decade_percent,id_station
start,end,,,,,,,,,,,,,,,,,
2017-10-22,2018-04-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,10,4,132.0,0.0,-100.0,nan
2018-10-22,2019-04-10,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,-100.0,0.0,NaN,10,4,132.0,0.0,-100.0,nan
2019-10-22,2020-04-10,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,-100.0,0.0,NaN,10,4,132.0,0.0,-100.0,nan
